# 🧠 ERP SQL Fine-tuning
Qwen2.5-coder modelini kendi ERP veritabanın için eğit!

**Gereksinimler:**
- Google Colab Pro (GPU için)
- Veya: T4/A100 GPU
- ~30 dakika

In [ ]:
# 1. Unsloth kur
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

In [ ]:
# 2. GPU kontrolü
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# 3. train.jsonl dosyasını yükle
# Dosyayı Colab'a yükle veya Google Drive'dan bağla
from google.colab import files
uploaded = files.upload()  # train.jsonl dosyasını seç

In [ ]:
# 4. Model yükle
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-Coder-7B-Instruct-bnb-4bit",
    max_seq_length=2048,
    load_in_4bit=True,
)

print("✓ Model yüklendi!")

In [ ]:
# 5. LoRA ekle
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
)

print("✓ LoRA eklendi!")

In [ ]:
# 6. Veri yükle
from datasets import load_dataset

dataset = load_dataset('json', data_files='train.jsonl', split='train')
print(f"✓ {len(dataset)} örnek yüklendi!")

# Format
def formatting_func(examples):
    texts = []
    for msgs in examples['messages']:
        text = tokenizer.apply_chat_template(msgs, tokenize=False)
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_func, batched=True)

In [ ]:
# 7. Eğit!
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        optim="adamw_8bit",
        output_dir="outputs",
        save_strategy="epoch",
    ),
)

print("Eğitim başlıyor... (~15-30 dakika)")
trainer.train()
print("\n✓ Eğitim tamamlandı!")

In [ ]:
# 8. Test et
FastLanguageModel.for_inference(model)

test_questions = [
    "bugün kaç sipariş girildi",
    "bu hafta hangi firmalardan sipariş verdik",
    "Daikin'e bu yıl ne kadar ödedik"
]

for q in test_questions:
    messages = [
        {"role": "system", "content": "Sen bir MSSQL uzmanısın. Türkçe soruyu SQL'e çevir."},
        {"role": "user", "content": q}
    ]
    
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs, max_new_tokens=300, temperature=0.1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print(f"\nSoru: {q}")
    print(f"SQL: {response.split('assistant')[-1].strip()}")

In [ ]:
# 9. GGUF olarak kaydet (Ollama için)
model.save_pretrained_gguf(
    "erp-sql-model",
    tokenizer,
    quantization_method="q4_k_m"
)

print("✓ GGUF kaydedildi!")

In [ ]:
# 10. İndir
!zip -r erp-sql-model.zip erp-sql-model/
files.download('erp-sql-model.zip')

print("""\n
✅ Model hazır!

Bilgisayarında kullanmak için:
1. erp-sql-model.zip'i aç
2. Modelfile oluştur (aşağıdaki hücreyi kopyala)
3. ollama create erp-sql -f Modelfile
4. ollama run erp-sql
""")

In [ ]:
# Modelfile içeriği (bunu kaydet)
print('''FROM ./erp-sql-model/unsloth.Q4_K_M.gguf

TEMPLATE """{{ if .System }}<|im_start|>system
{{ .System }}<|im_end|>
{{ end }}<|im_start|>user
{{ .Prompt }}<|im_end|>
<|im_start|>assistant
"""

SYSTEM """Sen bir MSSQL veritabanı uzmanısın. Türkçe soruyu SQL sorgusuna çeviriyorsun.
Kurallar:
- SADECE SQL yaz
- SELECT ile başla
- Firma adı için P.UNVAN kullan"""

PARAMETER temperature 0.1
PARAMETER stop "<|im_end|>"
''')